# Differential Privacy (DP) Aggregate Seeded Synthesizer

> Example based on [Synthetic Data Showcase - _pac-synth_](https://github.com/microsoft/synthetic-data-showcase/blob/main/packages/lib-pacsynth/samples/dp_aggregate_seeded_detailed_example.ipynb).

DP Aggregate Seeded synthesizer is a differentially private synthesizer that relies on DP Marginals to build synthetic data. It will compute DP Marginals (called aggregates) for your dataset using a specified `reporting length`, and synthesize data based on the computed aggregated counts.

> Aggregates will be computed for all lengths of attribute combination up to and including the `reporting length`.

## 1. Overview

### 1.1. Aggregate data generation with DP

Let's consider the following input as example:

| A   | B   | C   |
| --- | --- | --- |
| a1  | b1  | c1  |
| a1  | b2  | c1  |
| a2  |     | c2  |
| a2  | b2  | c1  |
| a1  | b2  |     |

The input data is assumed to be categorical and the domain will be inferred from the input dataset:

- `A` possible values are `a1,a2`
- `B` possible values are `b1,b2`
- `C` possible values are `c1,c2`

For a `reporting length=2`, the aggregates in the dataset above could be:

- 1-counts

  - `A:a1`: 3 + NOISE
  - `A:a2`: 2 + NOISE
  - `B:b1`: 1 + NOISE
  - `B:b2`: 3 + NOISE
  - `C:c1`: 3 + NOISE
  - `C:c2`: 1 + NOISE

- 2-counts:
  - `A:a1;C:c1`: 2 + NOISE
  - `A:a2;B:b2`: 1 + NOISE
  - `B:b1;C:c1`: 1 + NOISE
  - `A:a1;B:b1`: 1 + NOISE
  - `A:a1;B:b2`: 2 + NOISE
  - `B:b2;C:c1`: 2 + NOISE
  - `A:a2;C:c2`: 1 + NOISE
  - `B:b2;C:c2`: 0 + NOISE

Also, some spurious combinations might be created and reported to ensure differentially private guarantees - notice that `B:b2;C:c2` does not exist in the sensitive dataset, but it has been _fabricated_ and added to the output.

Similarly, some attribute combinations might be suppressed. For example, even though `A:a2;C:c1` exists in the sensitive dataset, it has not been reported as an aggregate.

### 1.2. Synthesis

Data will be then synthesized directly from the aggregates computed with differential privacy to produce synthetic data. Which will ensure the same DP guarantees to the synthetic data.


## 2. Imports and global config


In [1]:
import pandas as pd
import math

from snsynth.aggregate_seeded import (
    AggregateSeededSynthesizer,
    AccuracyMode,
    FabricationMode,
    AggregateSeededDataset,
)

from utils import gen_data_frame, ErrorReport

## 3. Generating an example data frame with random data

> `gen_data_frame` is just an utility to generate some example data (the code for it is in [`utils.py`](./utils.py))

To illustrate the library, let's start by creating an example data frame:


In [2]:
number_of_records_to_generate = 6000

sensitive_df = gen_data_frame(number_of_records_to_generate)

sensitive_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,2,1,4,0,1,0,1,0,0,0
1,2,1,2,0,0,1,1,0,1,1
2,1,1,,1,1,1,1,0,0,1
3,,3,,1,0,1,1,1,1,0
4,2,1,5,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5995,1,,8,0,1,1,0,1,1,0
5996,2,,10,0,1,0,1,0,0,1
5997,1,6,6,0,0,1,0,0,0,1
5998,2,6,8,0,0,0,0,1,1,0


## 4. Creating the sensitive dataset

### 4.1. Creation from constructor

The library uses an internal representation of the data to optimize execution time complexity.

If the data is already in the required raw format, you can call the constructor directly:


In [3]:
sensitive_raw_data = [
    # headers
    ["A", "B", "C", "D"],
    # records
    ["a1", "b1", "c1", "0"],
    ["a1", "b1", "c2", "0"],
    ["a2", "", "c2", "1"],
]
sensitive_dataset = AggregateSeededDataset(sensitive_raw_data)
sensitive_dataset.to_data_frame()

,A,B,C,D
0,a1,b1,c1,
1,a1,b1,c2,
2,a2,,c2,1


### 4.2. Negative value interpretation

The library distinguishes 'positive' attribute values that indicate the presence of specific sensitive data from 'negative' attribute values that indicate the absence of such data. By default, the integer zero (`0`) and the empty string (`""`) are not taken into account when creating and counting attribute combinations. Any columns where zero values are of interest (and thus sensitive) should be listed as `sensitive_zeros`, so they will be treated the same way as positive values.

> For more parameters see the library documentation - `help('pacsynth.Dataset')`.


In [4]:
sensitive_raw_data = [
    # headers
    ["A", "B", "C", "D"],
    # records
    ["a1", "b1", "c1", "0"],
    ["a1", "b1", "c2", "0"],
    ["a2", "", "c2", "1"],
]
sensitive_dataset = AggregateSeededDataset(sensitive_raw_data, sensitive_zeros=["D"])
sensitive_dataset.to_data_frame()

,A,B,C,D
0,a1,b1,c1,0
1,a1,b1,c2,0
2,a2,,c2,1


### 4.3. Creating sensitive dataset from a pandas data frame

For convenience, a method is provided to build a dataset from a pandas data frame:


In [5]:
sensitive_dataset = AggregateSeededDataset.from_data_frame(sensitive_df)
sensitive_dataset.to_data_frame()

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,2,1,4,,1,,1,,,
1,2,1,2,,,1,1,,1,1
2,1,1,,1,1,1,1,,,1
3,,3,,1,,1,1,1,1,
4,2,1,5,1,,,1,,,
...,...,...,...,...,...,...,...,...,...,...
5995,1,,8,,1,1,,1,1,
5996,2,,10,,1,,1,,,1
5997,1,6,6,,,1,,,,1
5998,2,6,8,,,,,1,1,


## 5. Generating the synthetic data

### 5.1. Defining synthesizer parameters

If you just want to create the synthesizer with default parameters:


In [6]:
synth = AggregateSeededSynthesizer()
print(synth.parameters)

{
  "reporting_length": 3,
  "epsilon": 4.0,
  "delta": null,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon_proportion": 0.005,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": null
}


However, this might not produce the optimal output for your dataset and downstream analysis tasks.

So you can tune configuration by changing synthesizer parameters accordingly:


In [7]:
# this explicitly outlines the default parameters
synth = AggregateSeededSynthesizer(
    reporting_length=3,
    epsilon=4.0,
    delta=1.0 / (math.log(len(sensitive_df)) * len(sensitive_df)),
    percentile_percentage=99,
    percentile_epsilon_proportion=0.01,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    number_of_records_epsilon_proportion=0.005,
    fabrication_mode=FabricationMode.uncontrolled(),
    empty_value="",
    weight_selection_percentile=95,
    use_synthetic_counts=False,
    aggregate_counts_scale_factor=1.0,
)
print(synth.parameters)

{
  "reporting_length": 3,
  "epsilon": 4.0,
  "delta": 0.000019158156689251674,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon_proportion": 0.005,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": 1.0
}


To continue with this example, let's set the parameters we care about for now:


In [8]:
reporting_length = 4

synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    fabrication_mode=FabricationMode.uncontrolled(),
    use_synthetic_counts=True,
)

### 5.2 Building the model and synthesizing data


In [9]:
synth.fit(sensitive_dataset)

# we could decide to use this or not as the sample number
protected_number_of_records = synth.get_dp_number_of_records()

print("Number of records protected with DP:", synth.get_dp_number_of_records())

# here if we do not specify the desired number of samples, the synthesizer will
# use all the available attributes based on the 1-counts to synthesize records
synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

# as an example, let's create a pandas data frame from the raw synthetic data
synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

Number of records protected with DP: 6015


,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,1,3,1,1,1,1,1,1,1
1,1,1,3,1,1,1,1,1,1,1
2,1,1,3,1,1,1,1,1,1,1
3,1,2,4,1,1,1,1,1,1,1
4,1,3,3,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
6010,,2,6,1,,,,,,
6011,,5,2,1,,,,,,
6012,,5,2,1,,,,,,
6013,1,2,8,,,,,,,


## 6. Generating/exporting aggregate data


In [10]:
# generate sensitive aggregates
sensitive_aggregates = synth.get_sensitive_aggregates(";")

# export the differentially private aggregates (internal to the synthesizer)
dp_aggregates = synth.get_dp_aggregates(";")

# generate aggregates from the synthetic data
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

# let's take a look at the DP aggregates
list(dp_aggregates.items())[:20]

[('H10:1;H1:2;H3:10;H7:1', 16),
 ('H2:6;H3:10;H5:1;H7:1', 53),
 ('H1:1;H3:6;H4:1;H8:1', 31),
 ('H1:1;H3:2;H5:1;H8:1', 37),
 ('H1:1;H3:3;H6:1;H8:1', 31),
 ('H2:5;H3:9;H6:1;H8:1', 32),
 ('H1:1;H3:7;H6:1', 95),
 ('H1:2;H2:5;H3:9;H9:1', 38),
 ('H2:6;H3:9;H8:1', 40),
 ('H10:1;H3:6;H8:1', 123),
 ('H1:2;H2:5;H4:1', 123),
 ('H10:1;H2:1;H3:3;H4:1', 4),
 ('H2:4;H3:8;H5:1;H9:1', 46),
 ('H10:1;H3:5;H4:1;H6:1', 83),
 ('H10:1;H1:1;H3:7;H8:1', 61),
 ('H2:1;H3:2;H6:1;H8:1', 40),
 ('H2:3;H3:7;H8:1', 5),
 ('H1:2;H2:6;H3:7;H4:1', 23),
 ('H10:1;H1:2;H5:1;H6:1', 255),
 ('H10:1;H3:7;H5:1', 122)]

## 7. Evaluating DP aggregates and DP synthetic data

This section is an example evaluation of both the DP aggregates and the DP synthetic data, as well as the influence of some synthesizer parameters in the DP aggregates and synthetic data.


### 7.1. Evaluating current results

> `ErrorReport` is just an example way to evaluate results (the code for it is [`utils.py`](./utils.py))

- **Count**: mean of the aggregate counts for the given length
- **Error**: mean of the `abs(sensitive_count - dp_aggregated_count)` or `abs(sensitive_count - synthetic_count)`
- **Suppressed %**: percentage of combinations present in the sensitive dataset, but not present in the aggregated/synthetic data
- **Fabricated %**: percentage of combinations that were reported in the aggregated/synthetic data, but do not exist in the sensitive dataset


In [11]:
sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**


In [12]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 13.60,0.00 %,0.00 %
1,2,425.60 +/- 13.22,0.00 %,5.43 %
2,3,149.25 +/- 13.38,0.00 %,5.42 %
3,4,56.58 +/- 12.20,3.27 %,1.95 %
4,All,111.65 +/- 12.58,2.21 %,3.09 %


**Sensitive Data vs. DP Synthetic Data**


In [13]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 83.24,0.00 %,0.00 %
1,2,425.60 +/- 61.72,0.00 %,4.57 %
2,3,149.25 +/- 33.57,0.00 %,4.32 %
3,4,56.58 +/- 14.89,3.35 %,1.45 %
4,All,111.65 +/- 23.14,2.27 %,2.41 %


### 7.2. Targeting less fabrication

Let's update the current synthesizer parameters to `minimize` fabrication:


In [14]:
synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_long_combinations(),
    fabrication_mode=FabricationMode.minimize(),
    use_synthetic_counts=True,
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,2,4,,1,1,1,1,1,1,1
1,2,4,,1,1,1,1,1,1,1
2,2,4,,1,1,1,1,1,1,1
3,2,4,,1,1,1,1,1,1,1
4,2,4,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
6010,2,5,7,,,,,,,
6011,2,5,7,,,,,,,
6012,2,5,7,,,,,,,
6013,2,5,7,,,,,,,


Evaluating again:


In [15]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ";")
dp_aggregates = synth.get_dp_aggregates(";")
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**


In [16]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 15.80,0.00 %,0.00 %
1,2,425.60 +/- 13.06,0.00 %,0.00 %
2,3,149.25 +/- 14.52,10.69 %,0.00 %
3,4,56.58 +/- 12.64,52.66 %,0.00 %
4,All,111.65 +/- 13.42,38.38 %,0.00 %


**Sensitive Data vs. DP Synthetic Data**


In [17]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 203.44,0.00 %,0.00 %
1,2,425.60 +/- 130.53,0.00 %,0.00 %
2,3,149.25 +/- 69.25,10.69 %,0.00 %
3,4,56.58 +/- 33.06,52.66 %,0.00 %
4,All,111.65 +/- 57.88,38.38 %,0.00 %


### 7.3. Prioritize short combinations

Let's update the current synthesizer parameters to `prioritize_short_combinations`:


In [18]:
synth = AggregateSeededSynthesizer(
    reporting_length=reporting_length,
    epsilon=4.0,
    accuracy_mode=AccuracyMode.prioritize_short_combinations(),
    fabrication_mode=FabricationMode.uncontrolled(),
    use_synthetic_counts=False,
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(protected_number_of_records)
synthetic_dataset = AggregateSeededDataset(synthetic_raw_data)

synthetic_df = AggregateSeededDataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
6010,,6,8,,,,,,,
6011,,6,8,,,,,,,
6012,,6,9,,,,,,,
6013,,6,9,,,,,,,


Evaluating again:


In [19]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ";")
dp_aggregates = synth.get_dp_aggregates(";")
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ";")

sensitive_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(";")): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**


In [20]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 3.68,0.00 %,0.00 %
1,2,425.60 +/- 9.05,0.00 %,7.52 %
2,3,149.25 +/- 21.66,0.88 %,8.17 %
3,4,56.58 +/- 26.52,27.23 %,4.24 %
4,All,111.65 +/- 23.53,18.65 %,5.72 %


**Sensitive Data vs. DP Synthetic Data**


In [21]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1381.00 +/- 69.12,0.00 %,0.00 %
1,2,425.60 +/- 115.51,0.00 %,6.28 %
2,3,149.25 +/- 73.08,0.99 %,6.26 %
3,4,56.58 +/- 44.58,28.53 %,2.90 %
4,All,111.65 +/- 59.04,19.56 %,4.22 %
